In [1]:
# add path to custom python code for accessing data lake and working with dataframes
import sys
OH_base_path = '/Users/markbills/Library/CloudStorage/OneDrive-Transformativ,LLC/Clients/Ovation Holdings'
sys.path.append(f'{OH_base_path}/src')

# Azure Data Lake libraries
import azure_data_lake_interface as adl

# Data analysis libraries
import pandas as pd

# Helper function libraries
import helper_functions as hf

In [2]:
# read spreadsheet
new_item_categories = pd.read_excel(f'{OH_base_path}/client_data/NewItemLevels.xlsx')

# make sure everything is interpreted as a string
new_item_categories = new_item_categories.astype(str)

In [3]:
# rename columns to match NetSuite naming convention
renames = {
    'Type': 'item_type',
    'Manufacturer': 'manufacturer',
    'Level 1': 'level_1_category',
    'Level 2': 'level_2_category',
    'Level 3': 'level_3_category',
    'Level 4': 'level_4_category',
    'Level 5': 'level_5_category',
    'Level 6': 'level_6_category',
    'Name': 'item_name',
    'Description': 'description',
}
new_item_categories.rename(columns=renames, inplace=True)

In [4]:
# attach to the data lake
config = hf.load_config("config/datalake_config.json", flush_cache=True)
service_client = adl.get_azure_service_client(config["blob_url"])
file_system_client = adl.get_azure_file_system_client(service_client, "consolidated")

# save in the data lake
adl.save_df_as_parquet_in_data_lake(new_item_categories, file_system_client, "enhanced/netsuite", "new_item_categories.parquet")